# Preprocessing

In [1]:
import pandas as pd
import re

In [2]:
def text_preprocessing(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text.encode('ascii', 'ignore').decode('ascii')
    if text.startswith("''"):
        text = text[1:-1]
    return text

In [3]:
data = pd.read_csv("/kaggle/input/fake-news-covid/prova")

display(data["id"])

0        1344763058888048643
1        1344756661567823872
2        1344733112454942721
3        1344770503014297602
4        1344719068520095744
                ...         
11425    1413112332385927169
11426    1413102602410156032
11427    1413142807024336909
11428    1413143486568681478
11429    1413129569662554117
Name: id, Length: 11430, dtype: int64

In [4]:
data_mis = pd.read_csv("/kaggle/input/fake-news-covid/VaxMisinfoData.csv")

display(data_mis["id"])

0        1344795424855642112
1        1344794858133860353
2        1344794822691983360
3        1344794752819077123
4        1344792070507134977
                ...         
15068    1413087751474397186
15069    1413087030578401283
15070    1413085793397186565
15071    1413085519710363648
15072    1413085365745774593
Name: id, Length: 15073, dtype: int64

In [5]:
data_final = data_mis.merge(data, on="id")

display(data_final["id"])

0        1344795424855642112
1        1344794858133860353
2        1344794822691983360
3        1344794752819077123
4        1344790296119422980
                ...         
11425    1413088663886573569
11426    1413087751474397186
11427    1413087030578401283
11428    1413085519710363648
11429    1413085365745774593
Name: id, Length: 11430, dtype: int64

In [6]:
#data_final["text"].head()

In [7]:
data_final["text"] = data_final["text"].apply(str).apply(lambda x: text_preprocessing(x))

#data_final["text"].head()

In [14]:
text_data = data_final["text"].values
print(len(text_data))

labels = data_final["is_misinfo"].values
print(len(labels))

11430
11430


Verifico bilanciamento delle etichette

In [9]:
labels_0 = [label for label in labels if label==0]
labels_1 = [label for label in labels if label==1]

print(len(labels_0))
print(len(labels_1))

7631
3799


# Preparing vectorstore

- Label 0: Notizie vere
- Label 1: Notizie false

In [10]:
%%capture --no-stderr
%pip install -U langchain_experimental langchain-ai21 langchain-pinecone langchain_community langchainhub langchain langchain-text-splitters

In [11]:
import os
os.environ["PINECONE_API_KEY"] = "94ef7896-1fae-44d3-b8d2-0bd6f5f664f5"
os.environ["AI21_API_KEY"] = "KlINkh5QKw3hG1b5Hr75YDO7TwGoQvzn"

In [15]:
from langchain.schema import Document

documents = [Document(page_content=row['text'], metadata={'label': row['is_misinfo']}) for index, row in data_final.iterrows()]

print(len(documents))

11430


In [13]:
import pandas as pd
from langchain_pinecone import PineconeVectorStore
from langchain_ai21 import AI21Embeddings

index_name = "entailment-test"

# Add to vectorDB
vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local", device="cuda"),
    embedding=AI21Embeddings(),
    index_name=index_name
)